In [ ]:
import torch
from torch import nn

# Seeding

In [ ]:
torch.manual_seed(0)

# Dataset Loading

In [ ]:
train = torch.rand(2000, 600, 600, 3)          
test = torch.rand(200, 600, 600, 3)            
train_y = torch.randint(0,1, (2000,))   
test_y = torch.randint(0, 1, (200,))

a0, a1

o1, o2

# Model Architeture

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.cnn_layer_1 = nn.Conv2d(in_channels=3, out_channels=256,kernel_size=3, stride=1)
        self.cnn_layer_2 = nn.Conv2d(in_channels=256, out_channels=32,kernel_size=3, stride=1)
        
        self.flatten = nn.Flatten()
        self.maxpool = nn.MaxPool2d(2)
        
        self.linear_layer_1 = nn.Linear(32*148*148, 512)  #1200 * 512
        self.linear_layer_2 = nn.Linear(512, 128) # 512*128
        self.linear_layer_3 = nn.Linear(128, 1)   #128*1
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(.2)
        # self.flatten = nn.Flatten()
        
    def forward(self, x):

        x = self.cnn_layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        print(x.shape)
        
        x = self.cnn_layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        print(x.shape)
        
        x = self.flatten(x)
        print(x.shape)
        
        x = self.linear_layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        
        x = self.linear_layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        
        x = self.linear_layer_3(x)
        logits = self.sigmoid(x)
        return logits

# Model Creation

In [ ]:
model = NeuralNetwork(1200)
print(model)

# Optimizer

In [ ]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# Model Training

In [ ]:
def trainModel(model, loss_fn, optimizer):
    model.train()

    batch = 128

    loss = 0

    for i in range(train.shape[0]):
      x, y = torch.reshape(train[i],(1,3,600,600)), torch.tensor([train_y[i]], dtype=torch.float)
    
      # Compute prediction error
      pred = model(x)[0]
      loss += loss_fn(pred,y)
      
      if i>0 and (i+1)%batch == 0:
          # Backpropagation
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          print(f'Training Loss: {loss.item():.4f}', end="\r")
          loss = 0
    print()

# Model testing

In [ ]:
def testModel(model, loss_fn):
    model.eval()

    size = test.shape[0]

    loss = 0
    with torch.no_grad():
      for i in range(test.shape[0]):
        x, y = torch.reshape(test[i],(1,3,600,600)), torch.tensor([test_y[i]], dtype=torch.float)
    
        # Compute prediction error
        pred = model(x)[0]
        loss += loss_fn(pred, y).item()
      
    loss /= size
    
    print(f'Testing Loss: {loss}')

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    trainModel(model, loss_fn, optimizer)
    testModel(model, loss_fn)
print("Done!")